In [6]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv
from tqdm import tqdm_notebook
import numpy as np

caps = webdriver.DesiredCapabilities().FIREFOX
caps["marionette"] = True
browser1 = webdriver.Firefox(executable_path=r'E:\Chrome\geckodriver.exe', capabilities=caps)
browser1.get("https://praktikum.yandex.ru/trainer/data-analyst/lesson/9bd17813-3d5c-47da-ab3c-097baced1979/task/03fbb4b3-ba91-4660-a05e-7b181f597407/")

# для ввода логина-пароля и очистки поля ввода запроса
time.sleep(15)

# ищем поле ввода sql-запроса и кнопку "play"
inputElement1 = browser1.find_elements_by_tag_name('textarea')[0]
button1 = browser1.find_elements_by_xpath("//button[@class='button button_has-hover-color button_size_m button_type_icon button_theme_light button_view_clear']")[0]

# пишем первый запрос и спим на всякий случай 
# (БД иногда плохо реагирует на почти одновременный ввод запроса и нажатие кнопки)
inputElement1.send_keys("select *\nfrom flights")
time.sleep(1)

# проверяем, есть ли вывод (когда вывод не подгрузился - кнопка ещё не активна)
# кнопку на "активность" и проверяем
try:
    button1.click()
except:
    try:
        time.sleep(1)
        button1.click()
    except:
        try:
            time.sleep(1)
            button1.click()
        except:
            pass
    
# ждем вывода результата    
time.sleep(5)

# парсим данные и ищем таблицу с результатом
parser1 = BeautifulSoup(browser1.page_source,"lxml")
table1 = parser1.find("table")

# пишем в файл
with open('flights.csv','w+', newline='') as csvfile1:
    writer1 = csv.writer(csvfile1, delimiter=',')
    
    for tr in table1.findAll("tr")[1:]:
        list_of_cells1 = list()
        for td in tr.findAll("td"):
            list_of_cells1.append(td.text)
        writer1.writerow(list_of_cells1)

# количество циклов в зависимости от общего количества строк в БД    
len_of_db = 65664 // 200

# запускаем цикл для ввода sql-запросов уже с offset и парсинга результатов
for i in tqdm_notebook(range(len_of_db)):
    prev_table1 = table1
    if i >= 1:
        chars_to_delete = len(str(i*200))
        for delete_count in range(chars_to_delete):
            inputElement1.send_keys(Keys.BACKSPACE) 
    i += 1
    if i > 1:
        inputElement1.send_keys("{}".format(i*200))
    else:
        inputElement1.send_keys("\noffset {}".format(i*200))
        
    time.sleep(0.5)
    try:
        button1.click()
    except:
        try:
            time.sleep(0.5)
            button1.click()
        except:
            try:
                time.sleep(0.5)
                button1.click()
            except:
                pass
    
    time.sleep(2)
    
    try:
        button1.click()
    except:
        try:
            time.sleep(1)
            button1.click()
            time.sleep(2)
        except:
            try:
                time.sleep(1)
                button1.click()
                time.sleep(2)
            except:
                pass
        
    parser1 = BeautifulSoup(browser1.page_source,"lxml")
    table1 = parser1.find("table")
    prev_counter = 0
    none_counter = 0
    
    while table1 == prev_table1 or table1 is None:
        if table1 == prev_table1:
            reason = 'повтор'
            prev_counter += 1
        elif table1 is None:
            reason = 'timeout'
            none_counter += 1
            
        print('Я тут, причина - {} в {} раз'.format(reason, [none_counter, prev_counter][reason == 'повтор']))
        
        time.sleep(2)
        try:
            button1.click()
            time.sleep(1)
        except:
            try:
                time.sleep(1)
                button1.click()
                time.sleep(1)
            except:
                pass
        parser1 = BeautifulSoup(browser1.page_source,"lxml")
        table1 = parser1.find("table")
    
    with open('flights.csv','a', newline='') as csvfile1:
        writer1 = csv.writer(csvfile1, delimiter=',')

        for tr in table1.findAll("tr")[1:]:
            list_of_cells1 = list()
            for td in tr.findAll("td"):
                list_of_cells1.append(td.text)
            writer1.writerow(list_of_cells1)